In [ ]:
# Updated solution
import math

class Canvas:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.data = [[' '] * width for _ in range(height)]

    def set_pixel(self, row, col, char='*'):
        self.data[row][col] = char

    def get_pixel(self, row, col):
        return self.data[row][col]
    
    def clear_canvas(self):
        self.data = [[' '] * self.width for _ in range(self.height)]
    
    def v_line(self, x, y, w, **kwargs):
        for i in range(y, y+w):
            if 0 <= i < self.height:
                self.set_pixel(x, i, **kwargs)

    def h_line(self, x, y, h, **kwargs):
        for i in range(x, x+h):
            if 0 <= i < self.width:
                self.set_pixel(i, y, **kwargs)
            
    def line(self, x1, y1, x2, y2, **kwargs):
        slope = (y2-y1) / (x2-x1) if x2-x1 != 0 else 0
        for x in range(x1, x2+1):
            y = int(slope * (x - x1) + y1)
            if 0 <= x < self.width and 0 <= y < self.height:
                self.set_pixel(x, y, **kwargs)
            
    def display(self):
        print("\n".join(["".join(row) for row in self.data]))

    def __repr__(self):
        return f'Canvas({self.width}, {self.height})'


class Shape:
    def area(self):
        raise NotImplementedError("Subclass must implement abstract method")

    def perimeter(self):
        raise NotImplementedError("Subclass must implement abstract method")

    def get_points(self):
        raise NotImplementedError("Subclass must implement abstract method")

    def is_inside(self, x, y):
        raise NotImplementedError("Subclass must implement abstract method")

    def overlaps(self, shape):
        raise NotImplementedError("Subclass must implement abstract method")

    def __repr__(self):
        raise NotImplementedError("Subclass must implement abstract method")


class Rectangle(Shape):
    def __init__(self, x, y, width, height):
        self.__x = x
        self.__y = y
        self.__width = width
        self.__height = height

    def area(self):
        return self.__width * self.__height

    def perimeter(self):
        return 2 * (self.__width + self.__height)

    def get_points(self):
        return [(self.__x + i, self.__y) for i in range(self.__width)] + \
               [(self.__x + self.__width, self.__y + i) for i in range(1, self.__height)] + \
               [(self.__x + i, self.__y + self.__height) for i in range(self.__width - 1, -1, -1)] + \
               [(self.__x, self.__y + i) for i in range(self.__height - 1, 0, -1)]

    def is_inside(self, x, y):
        return self.__x <= x <= self.__x + self.__width and \
               self.__y <= y <= self.__y + self.__height

    def overlaps(self, shape):
        if isinstance(shape, Rectangle):
            return not (self.__x + self.__width < shape.__x or
                        self.__x > shape.__x + shape.__width or
                        self.__y + self.__height < shape.__y or
                        self.__y > shape.__y + shape.__height)
        return False

    def __repr__(self):
        return f'Rectangle({self.__x}, {self.__y}, {self.__width}, {self.__height})'


class Circle(Shape):
    def __init__(self, x, y, radius):
        self.__x = x
        self.__y = y
        self.__radius = radius

    def area(self):
        return math.pi * self.__radius**2

    def perimeter(self):
        return 2 * math.pi * self.__radius

    def get_points(self):
        points = []
        for i in range(360):
            x = self.__x + self.__radius * math.cos(math.radians(i))
            y = self.__y + self.__radius * math.sin(math.radians(i))
            points.append((round(x), round(y)))
        return points

    def is_inside(self, x, y):
        return (x - self.__x)**2 + (y - self.__y)**2 <= self.__radius**2

    def overlaps(self, shape):
        if isinstance(shape, Circle):
            return math.sqrt((self.__x - shape.__x)**2 + (self.__y - shape.__y)**2) < self.__radius + shape.__radius
        return False

    def __repr__(self):
        return f'Circle({self.__x}, {self.__y}, {self.__radius})'

class Triangle(Shape):
    def __init__(self, x1, y1, x2, y2, x3, y3):
        self.__x1 = x1
        self.__y1 = y1
        self.__x2 = x2
        self.__y2 = y2
        self.__x3 = x3
        self.__y3 = y3

    def area(self):
        return abs((self.__x1*(self.__y2-self.__y3) + self.__x2*(self.__y3-self.__y1) + self.__x3*(self.__y1-self.__y2)) / 2)

    def perimeter(self):
        side1 = math.sqrt((self.__x2 - self.__x1)**2 + (self.__y2 - self.__y1)**2)
        side2 = math.sqrt((self.__x3 - self.__x2)**2 + (self.__y3 - self.__y2)**2)
        side3 = math.sqrt((self.__x1 - self.__x3)**2 + (self.__y1 - self.__y3)**2)
        return side1 + side2 + side3

    def get_points(self):
        return [(self.__x1 + i, self.__y1 + j) for i in range(int(self.__x2 - self.__x1) + 1) for j in range(int(self.__y2 - self.__y1) + 1)] + \
               [(self.__x2 + i, self.__y2 + j) for i in range(int(self.__x3 - self.__x2) + 1) for j in range(int(self.__y3 - self.__y2) + 1)] + \
               [(self.__x3 + i, self.__y3 + j) for i in range(int(self.__x1 - self.__x3) + 1) for j in range(int(self.__y1 - self.__y3) + 1)]

    def is_inside(self, x, y):
        # vectors
        v0 = [self.__x3 - self.__x1, self.__y3 - self.__y1]
        v1 = [self.__x2 - self.__x1, self.__y2 - self.__y1]
        v2 = [x - self.__x1, y - self.__y1]

        
        dot00 = v0[0]*v0[0] + v0[1]*v0[1]
        dot01 = v0[0]*v1[0] + v0[1]*v1[1]
        dot02 = v0[0]*v2[0] + v0[1]*v2[1]
        dot11 = v1[0]*v1[0] + v1[1]*v1[1]
        dot12 = v1[0]*v2[0] + v1[1]*v2[1]

        
        inv_denom = 1 / (dot00 * dot11 - dot01 * dot01)
        u = (dot11 * dot02 - dot01 * dot12) * inv_denom
        v = (dot00 * dot12 - dot01 * dot02) * inv_denom
        return (u >= 0) and (v >= 0) and (u + v < 1)

    def overlaps(self, shape):
        for point in shape.get_points():
            x, y = point
            if self.is_inside(x, y):
                return True
        return False

    def __repr__(self):
        return f'Triangle({self.__x1}, {self.__y1}, {self.__x2}, {self.__y2}, {self.__x3}, {self.__y3})'


class Paint:
    def __init__(self, canvas):
        self.canvas = canvas

    def draw_shape(self, shape, char='*'):
        for point in shape.get_points():
            x, y = point
            if 0 <= x < self.canvas.width and 0 <= y < self.canvas.height:
                self.canvas.set_pixel(y, x, char)


class RasterDrawing:
    def __init__(self, canvas):
        self.canvas = canvas
        self.shapes = []

    def add_shape(self, shape):
        self.shapes.append(shape)

    def draw(self):
        paint = Paint(self.canvas)
        for shape in self.shapes:
            paint.draw_shape(shape)

    def save(self, filename):
        with open(filename, 'w') as file:
            for shape in self.shapes:
                file.write(repr(shape) + '\n')

    @classmethod
    def load(cls, filename):
        with open(filename, 'r') as file:
            lines = file.readlines()
            canvas = None
            drawing = cls(None)
            for line in lines:
                obj = eval(line.strip())
                if isinstance(obj, Canvas):
                    canvas = obj
                elif canvas:
                    drawing.add_shape(obj)
                else:
                    raise ValueError("Canvas not found in the file.")
            drawing.canvas = canvas
            return drawing


class CompoundShape(Shape):
    def __init__(self, shapes):
        self.shapes = shapes

    def area(self):
        return sum(shape.area() for shape in self.shapes)

    def perimeter(self):
        return sum(shape.perimeter() for shape in self.shapes)

    def get_points(self):
        points = []
        for shape in self.shapes:
            points.extend(shape.get_points())
        return points

    def is_inside(self, x, y):
        for shape in self.shapes:
            if shape.is_inside(x, y):
                return True
        return False

    def overlaps(self, shape):
        for self_shape in self.shapes:
            if self_shape.overlaps(shape):
                return True
        return False

10. Copy the Canvas class from lecture to in a python file creating a paint module. Copy your classes from above into the module and implement paint functions. Implement a CompoundShape class. Create a simple drawing demonstrating that all of your classes are working.

In [9]:
# In the new Python script
import cpaint

# Create a canvas
canvas = cpaint.Canvas(20, 10)

# Create shape
rectangle = cpaint.Rectangle(2, 2, 10, 5)

# Draw shapes
drawing = cpaint.RasterDrawing(canvas)
drawing.add_shape(rectangle)
drawing.draw()


# Display the canvas
canvas.display()

                    
                    
  **********        
  *         *       
  *         *       
  *         *       
  *         *       
  **********        
                    
                    


In [24]:
import cpaint

# Create a canvas
canvas = cpaint.Canvas(15, 15)

# Create shape
circle = cpaint.Circle(2, 9, 2)

# Draw shapes
drawing = cpaint.RasterDrawing(canvas)
drawing.add_shape(circle)
drawing.draw()


# Display the canvas
canvas.display()

               
               
               
               
               
               
               
 ***           
** **          
*   *          
** **          
 ***           
               
               
               


In [30]:
import cpaint

# Create a canvas
canvas = cpaint.Canvas(20, 20)

# Create shape
triangle = cpaint.Triangle(1, 9,9,15,15,0)

# Draw shapes
drawing = cpaint.RasterDrawing(canvas)
drawing.add_shape(triangle)
drawing.draw()


# Display the canvas
canvas.display()

                    
                    
                    
                    
                    
                    
                    
                    
                    
 *********          
 *********          
 *********          
 *********          
 *********          
 *********          
 *********          
                    
                    
                    
                    


11. Create a RasterDrawing class. Demonstrate that you can create a drawing made of several shapes, paint the drawing, modify the drawing, and paint it again.

In [10]:
# Python script/module
import cpaint

# Create canvas
canvas = cpaint.Canvas(20, 10)

# Create shapes
rectangle = cpaint.Rectangle(2, 2, 10, 5)
triangle = cpaint.Triangle(0,2,2,5,4,8)

# Draw shape
drawing = cpaint.RasterDrawing(canvas)
drawing.add_shape(rectangle)
drawing.draw()
drawing2 = cpaint.RasterDrawing(canvas)
drawing2.add_shape(triangle)
drawing2.draw()

# Display
canvas.display()

                    
                    
************        
***         *       
***         *       
*****       *       
  ***       *       
  **********        
  ***               
                    


12. Implement the ability to load/save raster drawings and demonstate that your method works. One way to implement this ability:
Overload __repr__ functions of all objects to return strings of the python code that would construct the object.

In the save method of raster drawing class, store the representations into the file.

Write a loader function that reads the file and uses eval to instantiate the object.